In [ ]:
# Import packages
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os

In [ ]:
# Import facilities dataset
fac = pd.read_csv("./Data/facilities.csv")
fac.head()

In [ ]:
# Check number of missing values per variable
for col in fac.columns:
    missings = len(fac[col][fac[col].isnull()]) / float(len(fac))
    print(col, missings)

# All sales_* columns have a very high number of missing values indicating a data quality issue. 

In [ ]:
# Further investigation of missing values in facilities columns

# we will exclude sales_* (because of DQ issue) and determine which columns are metadata
exclude_prefix = 'sales_'
metadata = {'station_id', 'station', 'name', 'street', 'zip', 'city'}
cols_to_check = [
    c for c in fac.columns
    if not c.startswith(exclude_prefix) and c not in metadata
]

print(f"Number of columns checked: {len(cols_to_check)}")
print("Some of the checked columns:", cols_to_check[:12])

# make mask for rows with at least one NaN in those columns
nan_mask = fac[cols_to_check].isna().any(axis=1)

# make results table
id_cols = [c for c in ['station_id','station','name'] if c in fac.columns]
result_cols = id_cols + cols_to_check

nan_stations = fac.loc[nan_mask, result_cols].copy()

# add column to table that counts how many NaN's there are per station for the checked columns
nan_stations['n_missing'] = fac.loc[nan_mask, cols_to_check].isna().sum(axis=1)

nan_per_column = nan_stations[cols_to_check].isna().sum().sort_values(ascending=False).to_frame(name='n_missing_in_subset')

# overview
print("Number of stations with at least 1 NaN:", nan_stations.shape[0])
display(nan_stations.head(50)) 
display(nan_per_column)   

In [ ]:
# From the above, we see that the stations with missing facility data contain no information in all facility columns.
# Therefore, we will remove all stations with missing facility data from the dataset to perform our analysis.
# Additionally, we exclude all sales_* columns from the resulting dataframe.
sales_cols = [col for col in fac.columns if col.startswith('sales_')]
cols_to_keep = [col for col in fac.columns if col not in sales_cols]
fac_no_missing_facilities = fac.loc[~nan_mask, cols_to_keep].copy()
fac_no_missing_facilities.head()

In [ ]:
# We transform disabled_parking_spots to a binary indicator
fac_no_missing_facilities["disabled_parking_ind"] = (fac["disabled_parking_spots"].fillna(0) > 0).astype(float)

# Binary columns
binary_cols = ["ticket_vending_machine","luggage_lockers","free_parking","taxi","bicycle_spots","blue-bike","bus","tram","metro","wheelchair_available","ramp","disabled_parking_ind","elevated_platform","escalator_up","escalator_down","elevator_platform","audio_induction_loop"]

# Sum over binary columns + disabled_parking_ind
fac_no_missing_facilities["n_facilities"] = fac_no_missing_facilities[binary_cols].sum(axis=1)
fac_no_missing_facilities


In [ ]:
# Import travelers dataset
travelers = pd.read_csv("./Data/travelers.csv", sep=";", index_col=0)
# Rename for convenience
travelers = travelers.rename({"Station": "station",
                                    "Avg number of travelers in the week": "week",
                                    "Avg number of travelers on Saturday": "saturday",
                                    "Avg number of travelers on Sunday": "sunday"}, axis=1)
travelers.head()

In [ ]:
# Check number of missing values per variable
for col in travelers.columns:
    missings = len(travelers[col][travelers[col].isnull()]) / float(len(travelers))
    print(col, missings)

In [ ]:
# Further inspection on Wikipedia and the NMBS website reveal that there are no train rides on these dates for these stations. 
# For example, Baasrode-Zuid & Buda only have train rides during the week and none in the weekend. 
# Therefore, we will impute every missing value with zero.
travelers['week'] = travelers['week'].fillna(0)
travelers['saturday'] = travelers['saturday'].fillna(0)
travelers['sunday'] = travelers['sunday'].fillna(0)

# Show
travelers.head()

In [ ]:
# Create total travelers over the week
travelers["week_total"] = 5 * travelers["week"] + travelers["saturday"] + travelers["sunday"]

In [ ]:
# Get avg travelers per day (including weekends)
travelers["avg_day"] = travelers["week_total"] / float(7)
travelers

In [ ]:
# Import stations dataset
stations = pd.read_csv("./Data/stations.csv")
stations.head()

In [ ]:
stations["daily_trains"] = stations["avg_stop_times"]
stations["daily_trains"]

In [ ]:
# Merge datasets on station_id and name
from unidecode import unidecode
fac_no_missing_facilities['name_clean'] = fac_no_missing_facilities['name'].apply(lambda x: unidecode(str(x)).lower().strip())
travelers['station_clean'] = travelers['station'].apply(lambda x: unidecode(str(x)).lower().strip())
df = fac_no_missing_facilities.merge(stations[['station_id', 'daily_trains']], on='station_id', how='left')
df = df.merge(travelers[['station_clean', 'avg_day']],
              left_on='name_clean', right_on='station_clean', how='left')
df

In [ ]:
# Regression: number of facilities on number of trains and number of travelers
reg_df = df[['n_facilities', 'daily_trains', 'avg_day']].dropna()
X = reg_df[['daily_trains', 'avg_day']]
y = reg_df['n_facilities']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
incidents = pd.read_csv("./Data/incidents.csv", sep=";", index_col=0)


In [ ]:
# Display all rows where 'place' is '-'
dash_rows = incidents[incidents['Place'] == "-"]
display(dash_rows)

In [ ]:
# Remove duplicate columns 'Place.1' en 'Place.2'
if 'Place.1' in incidents.columns:
    incidents = incidents.drop(columns=['Place.1'])
if 'Place.2' in incidents.columns:
    incidents = incidents.drop(columns=['Place.2'])
incidents.head()

In [ ]:
# Missing value analysis in incidents dataset
for col in incidents.columns:
    missings = len(incidents[col][incidents[col].isnull()]) / float(len(incidents))
    print(col, missings)

In [ ]:
incidents.columns = incidents.columns.str.lower()

In [ ]:
# threshold for extreme delays
threshold = incidents['minutes of delay'].quantile(0.9)
extreme_incidents = incidents[incidents['minutes of delay'] >= threshold]
extreme_incidents

In [ ]:
# Analysis of extreme delays by incident type
type_summary = (extreme_incidents.groupby('incident description', as_index = False).agg(n_extreme=('minutes of delay', 'count'),
        avg_delay=('minutes of delay', 'mean'),
        total_delay=('minutes of delay', 'sum'),
        avg_cancellations=('number of cancelled trains', 'mean')).sort_values('total_delay', ascending = False))
display(type_summary.head(10))

In [195]:
# Analysis of extreme delays by location by looking at incident total delay 
location_summary = (extreme_incidents.groupby('place', as_index=False)
    .agg(
        n_extreme=('minutes of delay', 'count'),
        avg_delay=('minutes of delay', 'mean'),
        total_cancelled=('number of cancelled trains', 'sum')
    )
    .sort_values('avg_delay', ascending=False)
)

display(location_summary.head(10))

,place,n_extreme,avg_delay,total_cancelled
0,-,13,15472.538462,6801
9,BRUSSEL-NOORD,6,10166.666667,1626
2,ANDERLECHT,1,9671.000000,207
7,BRUSSEL-CENTRAAL,5,7283.800000,657
16,GEMBLOUX,3,6692.000000,503
1,AALTER,1,6689.000000,207
5,ANTWERPEN-CENTRAAL,1,6640.000000,154
4,ANTWERPEN-BERCHEM,1,6622.000000,124
34,VELTEM,1,6528.000000,171
40,ZAVENTEM,2,6026.000000,238


In [ ]:
extreme_incidents['place_clean'] = extreme_incidents['place'].str.lower().str.strip()
travelers['station_clean'] = travelers['station'].str.lower().str.strip()

impact = extreme_incidents.merge(
    travelers[['station_clean', 'week_total']],
    left_on='place_clean', right_on='station_clean', how='left'
)

impact["affected_passengers"] = impact["week_total"]

impact_summary = (
    impact.groupby('place', as_index=False)
    .agg(
        n_extreme=('minutes of delay', 'count'),
        avg_delay=('minutes of delay', 'mean'),
        total_delay=('minutes of delay', 'sum'),
        total_cancelled=('number of cancelled trains', 'sum'),
        avg_travelers=('affected_passengers', 'mean')
    )
)

# ImpactScore ​= Totaal aantal minuten vertraging op station i × Gemiddeld aantal reizigers op station i
# Toont waar veel reizigers kunnen worden beïnvloed door vertragingen.
impact_summary['passenger_delay_index'] = impact_summary['total_delay'] * impact_summary['avg_travelers']

impact_summary = impact_summary.sort_values('passenger_delay_index', ascending=False)

display(impact_summary.head(10))

In [ ]:
import matplotlib.pyplot as plt

top = priority.head(10)
plt.barh(top['place'], top['passenger_delay_index'])
plt.xlabel('Passenger Delay Index (delay × travelers)')
plt.ylabel('Station')
plt.title('Top 10 Stations with biggest impact on travelers in case of extreme delays')
plt.gca().invert_yaxis()
plt.show()
